In [154]:
from importlib import reload
import data.loader as loader_module
import evaluation.predict as predict_module
import evaluation.eval as eval_module
import data.squad_loader as squad_loader_module
import json


reload(predict_module)
reload(eval_module)
reload(loader_module)
reload(squad_loader_module)

from evaluation.predict import ModelPredictor
from evaluation.eval import evaluate_generation, evaluate_classification, get_confusion_matrix
from data.loader import Dataset, get_dataset
from data.squad_loader import get_squad_dataset

## Load dataset

In [244]:
full_evaluation = True
percent_types = (0.5, 0.25, 0.25)
train_set, test_set = get_squad_dataset(
    500 if full_evaluation else 50,
    split_percentage=1.0,
    percent_types=percent_types,
    types=("answerable", "unanswerable_base", "unanswerable_ner"),
)
print(len(train_set))
print(train_set[-1])
print(list(train_set.get_column("modified")))
print(list(train_set.get_column("is_impossible")))
train_set.data[-2:]


500
{'context': "Encouraged by these successes, Nintendo soon turned its attention to the North American market. Nintendo entered into negotiations with Atari to release the Famicom under Atari’s name as the name Nintendo Advanced Video Gaming System. The deal was set to be finalized and signed at the Summer Consumer Electronics Show in June 1983. However, Atari discovered at that show that its competitor Coleco was illegally demonstrating its Coleco Adam computer with Nintendo's Donkey Kong game. This violation of Atari's exclusive license with Nintendo to publish the game for its own computer systems delayed the implementation of Nintendo's game console marketing contract with Atari. Atari's CEO Ray Kassar was fired the next month, so the deal went nowhere, and Nintendo decided to market its system on its own.g[›]", 'plausible_answers': [{'text': 'June 1983', 'answer_start': 322}], 'question': ' When was the Summer Consumer Electronics Show rejected?', 'id': '5ad2a2b9d7d075001a429d1b

[{'context': 'Some species of marine animals exist and rely, directly or indirectly, on the phytoplankton. Antarctic sea life includes penguins, blue whales, orcas, colossal squids and fur seals. The emperor penguin is the only penguin that breeds during the winter in Antarctica, while the Adélie penguin breeds farther south than any other penguin. The rockhopper penguin has distinctive feathers around the eyes, giving the appearance of elaborate eyelashes. King penguins, chinstrap penguins, and gentoo penguins also breed in the Antarctic.',
  'question': 'What feature does the rockhopper penguin have around its eyes?',
  'id': '570df3470dc6ce1900204d4a',
  'answers': [{'text': 'feathers', 'answer_start': 376}],
  'is_impossible': False},
 {'context': "Encouraged by these successes, Nintendo soon turned its attention to the North American market. Nintendo entered into negotiations with Atari to release the Famicom under Atari’s name as the name Nintendo Advanced Video Gaming System. Th

In [237]:
for i in range(min(len(train_set), 10)):
    print(train_set.data[i])


{'context': "During World War II, Hayek began the ‘Abuse of Reason’ project. His goal was to show how a number of then-popular doctrines and beliefs had a common origin in some fundamental misconceptions about the social science. In his philosophy of science, which has much in common with that of his good friend Karl Popper, Hayek was highly critical of what he termed scientism: a false understanding of the methods of science that has been mistakenly forced upon the social sciences, but that is contrary to the practices of genuine science. Usually, scientism involves combining the philosophers' ancient demand for demonstrative justification with the associationists' false view that all scientific explanations are simple two-variable linear relationships.", 'question': "Whose beliefs in scientific philosophy were similar to Hayek's?", 'id': '572a8062f75d5e190021fb10', 'answers': [{'text': 'Karl Popper', 'answer_start': 301}], 'is_impossible': False}
{'context': 'In May 2008, Dell reache

## Predict

In [238]:
#model_name = "davinci-002"
model_name = "gpt-3.5-turbo-instruct"
model = ModelPredictor(model_name)

In [93]:
def extract_unknown(text):
    if "no answer" in text.lower():
        return "" # evaluation process wants empty string for predictions of unanswerable questions
    else:
        return text


In [94]:
"""CONTEXT: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
QUESTION: What century did the Normans first gain their separate identity?
ANSWER(concise): 10th

CONTEXT: Building construction is the process of adding structure to real property or construction of buildings. The majority of building construction jobs are small renovations, such as addition of a room, or renovation of a bathroom. Often, the owner of the property acts as laborer, paymaster, and design team for the entire project. Although building construction projects typically include various common elements, such as design, financial, estimating and legal considerations, many projects of varying sizes reach undesirable end results, such as structural collapse, cost overruns, and/or litigation. For this reason, those with experience in the field make detailed plans and maintain careful oversight during the project to ensure a positive outcome.
QUESTION: Who acts as laborer, paymaster, and design team for a renovation project?
ANSWER(concise): the owner

CONTEXT: Pathogens can rapidly evolve and adapt, and thereby avoid detection and neutralization by the immune system; however, multiple defense mechanisms have also evolved to recognize and neutralize pathogens. Even simple unicellular organisms such as bacteria possess a rudimentary immune system, in the form of enzymes that protect against bacteriophage infections. Other basic immune mechanisms evolved in ancient eukaryotes and remain in their modern descendants, such as plants and invertebrates. These mechanisms include phagocytosis, antimicrobial peptides called defensins, and the complement system. Jawed vertebrates, including humans, have even more sophisticated defense mechanisms, including the ability to adapt over time to recognize specific pathogens more efficiently. Adaptive (or acquired) immunity creates immunological memory after an initial response to a specific pathogen, leading to an enhanced response to subsequent encounters with that same pathogen. This process of acquired immunity is the basis of vaccination.
QUESTION: The immune systems of bacteria have enzymes that protect against infection by what kind of cells?
ANSWER(concise): bacteriophage

CONTEXT: {context}
QUESTION: {question}
ANSWER(concise): """ 

'Given the following context:\nCONTEXT: {context}\nAnswer the following question:\nQUESTION: {question}\nANSWER: '

In [189]:
prompt_template = """CONTEXT: {context}
QUESTION: {question}
ANSWER(concise): """ 

results_generate, original_preds = model.generate(
    dataset=train_set,
    prompt_template=prompt_template,
    extract_function=lambda text: text, #extract_unknown, 
    num_samples=1,
    max_tokens=50,
)
results_generate, original_preds

([' Falaise and then Rouen.',
  " Sony's music publishing operations.",
  'The University of Houston System owns and holds broadcasting licenses for KUHT, KUHF, and KUHA. ',
  '2nd century AD',
  ' 200 trucks.',
  'Birds are immune to avian influenza.',
  ' The need for access to a large quantity of information.',
  'Gold-alloy touch surfaces have natural intrinsic properties to destroy a wide range of microorganisms.',
  '109 million.',
  'Mount Oread.',
  ' Musharraf',
  '\nLauryn Hill.',
  '\nHeavy fuel oil from the freighter MS Oliva.',
  'The composition of mass changes into energy.',
  'Embarrassment to the FBI.',
  ' Rationalism, empiricism, structuralism, poststructuralism, and phenomenology.',
  'Die casts made of ceramics and cement.',
  '1,307,402',
  ' The House of the Black Madonna.',
  'At age seven or eight.',
  ' Half of the money is divided equally between the clubs.',
  ' The largest neutrino telescope in the world. ',
  ' Several times a week.',
  'Southeast.',
  '\n

In [105]:
"""CONTEXT: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
QUESTION: What century did the Normans first gain their separate identity?
Is the answer to the QUESTION available in the CONTEXT?
ANSWER(yes/no): yes

CONTEXT: Building construction is the process of adding structure to real property or construction of buildings. The majority of building construction jobs are small renovations, such as addition of a room, or renovation of a bathroom. Often, the owner of the property acts as laborer, paymaster, and design team for the entire project. Although building construction projects typically include various common elements, such as design, financial, estimating and legal considerations, many projects of varying sizes reach undesirable end results, such as structural collapse, cost overruns, and/or litigation. For this reason, those with experience in the field make detailed plans and maintain careful oversight during the project to ensure a positive outcome.
QUESTION: What are some examples of legal considerations for an addition?
Is the answer to the QUESTION available in the CONTEXT?
ANSWER(yes/no): no

CONTEXT: Pathogens can rapidly evolve and adapt, and thereby avoid detection and neutralization by the immune system; however, multiple defense mechanisms have also evolved to recognize and neutralize pathogens. Even simple unicellular organisms such as bacteria possess a rudimentary immune system, in the form of enzymes that protect against bacteriophage infections. Other basic immune mechanisms evolved in ancient eukaryotes and remain in their modern descendants, such as plants and invertebrates. These mechanisms include phagocytosis, antimicrobial peptides called defensins, and the complement system. Jawed vertebrates, including humans, have even more sophisticated defense mechanisms, including the ability to adapt over time to recognize specific pathogens more efficiently. Adaptive (or acquired) immunity creates immunological memory after an initial response to a specific pathogen, leading to an enhanced response to subsequent encounters with that same pathogen. This process of acquired immunity is the basis of vaccination.
QUESTION: The immune systems of bacteria have enzymes that protect against infection by what kind of cells?
Is the answer to the QUESTION available in the CONTEXT?
ANSWER(yes/no): yes

CONTEXT: {context}
QUESTION: {question}
Is the answer to the QUESTION available in the CONTEXT?
ANSWER(yes/no): """ 

In [245]:
def extract_yes_or_no(text):
    if "yes" in text.lower():
        return "Yes"
    elif "no" in text.lower():
        return "No"
    else:
        return None

classify_with_generation_template = """CONTEXT: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
QUESTION: What century did the Normans first gain their separate identity?
Is the answer to the QUESTION available in the CONTEXT?
ANSWER(yes/no): yes

CONTEXT: Building construction is the process of adding structure to real property or construction of buildings. The majority of building construction jobs are small renovations, such as addition of a room, or renovation of a bathroom. Often, the owner of the property acts as laborer, paymaster, and design team for the entire project. Although building construction projects typically include various common elements, such as design, financial, estimating and legal considerations, many projects of varying sizes reach undesirable end results, such as structural collapse, cost overruns, and/or litigation. For this reason, those with experience in the field make detailed plans and maintain careful oversight during the project to ensure a positive outcome.
QUESTION: What are some examples of legal considerations for an addition?
Is the answer to the QUESTION available in the CONTEXT?
ANSWER(yes/no): no

CONTEXT: Pathogens can rapidly evolve and adapt, and thereby avoid detection and neutralization by the immune system; however, multiple defense mechanisms have also evolved to recognize and neutralize pathogens. Even simple unicellular organisms such as bacteria possess a rudimentary immune system, in the form of enzymes that protect against bacteriophage infections. Other basic immune mechanisms evolved in ancient eukaryotes and remain in their modern descendants, such as plants and invertebrates. These mechanisms include phagocytosis, antimicrobial peptides called defensins, and the complement system. Jawed vertebrates, including humans, have even more sophisticated defense mechanisms, including the ability to adapt over time to recognize specific pathogens more efficiently. Adaptive (or acquired) immunity creates immunological memory after an initial response to a specific pathogen, leading to an enhanced response to subsequent encounters with that same pathogen. This process of acquired immunity is the basis of vaccination.
QUESTION: The immune systems of bacteria have enzymes that protect against infection by what kind of cells?
Is the answer to the QUESTION available in the CONTEXT?
ANSWER(yes/no): yes

CONTEXT: {context}
QUESTION: {question}
Is the answer to the QUESTION available in the CONTEXT?
ANSWER(yes/no): """ 

results_classify, original_preds_class = model.generate(
    dataset=train_set,
    prompt_template=classify_with_generation_template,
    extract_function=extract_yes_or_no, 
    num_samples=1,
    max_tokens=5,
)
results_classify, original_preds_class

## Eval

In [39]:
#list(train_set.get_column("answers")), results_generate, results_classify

In [175]:
def save_results(prompt, exact_score, f1_score, model_name, percent_types,full_results=False):
    with open("results/results_squad.txt", "a") as f:
        prepared_prompt = prompt.replace("\n", r"\n")
        f.write(f"{percent_types}\t{exact_score}\t{f1_score}\t{model_name}\t{full_results}\t{prepared_prompt}\n")

In [120]:
# def load_results():
#     with open("results/results_squad.txt", "r") as f:
#         lines = f.readlines()
#         lines = [line.strip() for line in lines]
#         results = [line.split("\t") for line in lines]
#         return results
# results_squad = load_results()

# mod_res = [res[1:] + (['False'] if len(res)==4 else []) + [res[0] + "\n"] for res in results_squad]
# mod_res

# # save mod_res to file
# with open("results/results_squad.txt", "w") as f:
#     for res in mod_res:
#         f.write("\t".join(res))

In [190]:
import evaluation.squad_eval as squad_eval_module
reload(squad_eval_module)
from evaluation.squad_eval import get_raw_scores_mod

preds = dict(zip(train_set.get_column("id"), results_generate))
raws_scores, f1_scores = get_raw_scores_mod(train_set, preds)

exact_score = sum(raws_scores.values()) / len(raws_scores)
f1_score = sum(f1_scores.values()) / len(f1_scores)
print("Exact score:", exact_score)
print("F1 score:", f1_score)
save_results(
    prompt_template,
    exact_score,
    f1_score,
    model_name,
    percent_types,
    full_results=full_evaluation,
)

Exact score: 0.28
F1 score: 0.37196336996337


In [157]:
to_choice_mapping = {
    True: "No", # true is_impossible
    False: "Yes", # false is_impossible
}
results_classify, list(train_set.get_type(to_choice_mapping))

(['Yes',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'No',
  'Yes',
  'Yes',
  None,
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'No',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes'],
 ['Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes',
  'Yes'])

In [231]:
import numpy as np
from typing import List, Dict

def get_binary_confusion_matrix(
    labels: List[str],
    predictions: List[str],
) -> Dict[str, Dict[str, int]]:
    labels = np.array(labels)
    predictions = np.array(predictions)
    tp = (labels == predictions) & (labels == "Yes")
    fp = (labels != predictions) & (predictions == "Yes")
    tn = (labels == predictions) & (labels == "No")
    fn = (labels != predictions) & (predictions == "No")
    return {
        "tp": int(sum(tp)),
        "fp": int(sum(fp)),
        "tn": int(sum(tn)),
        "fn": int(sum(fn)),
    }
    

In [ ]:
import json

correct, accuracy =evaluate_classification(
    train_set,
    results_classify,  # type: ignore
    to_choice_mapping=to_choice_mapping,
)
confussion_matrix = get_binary_confusion_matrix(list(train_set.get_type(to_choice_mapping)), results_classify) 
print(confussion_matrix )
print(accuracy)

save_results(
    classify_with_generation_template,
    accuracy,
    json.dumps(confussion_matrix),
    model_name,
    percent_types,
    full_results=full_evaluation,
)

{'tp': 22, 'fp': 16, 'tn': 8, 'fn': 4}
0.6


In [199]:
get_confusion_matrix(
    predict=["unknown", "10", "11", "9", "unknown"],
    original=["unknown", "9", "11", "unknown", "unknown"],
    unknown_value = "unknown"
)

(array([[1., 0.],
        [0., 2.],
        [1., 1.]]),
 0)